In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import AnimalShelter 
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Pass username and password to CRUD Python module name
username = "aacuser"
password = "SNHU1234"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

#########################
# Dashboard Layout / View
#########################

app = JupyterDash(__name__)

# Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.B(html.H1('Created By: Christopher King'))),
    html.Hr(),
        # Radio Buttons for filter selection
        dcc.RadioItems(
            id='filter-type',
           options=[
               {'label': 'Water Rescue', 'value': 'Water'},
               {'label': 'Mountain Rescue', 'value': 'Mountain'},
               {'label': 'Disaster Rescue', 'value': 'Disaster'},
               {'label': 'Reset', 'value': 'Reset'},
           ],
           value='Reset' # Default to no filter
        ),          
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10, # Limit records per page to 10
        row_selectable = 'single', # Allow rows to be selected
        sort_action='native', # Enable sorting
        page_action='native', # Enable pagination
        filter_action='native', #Enable filtering   
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
                html.Div(
                    id='graph-id',
                    className='col s12 m6',

                    ),
                html.Div(
                    id='map-id',
                    className='col s12 m6',
                )
            ])
    ])

#############################################
# Interaction Between Components / Controller
#############################################
   
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
# Define different query based on filter selected
    query = {}
    # Water Dog filter
    if filter_type == 'Water':
        query = {
            "animal_type": "Dog",
            "sex_upon_outcome": "Intact Female",
            "$and": [
                {"$or": [
                    {"breed": "Labrador Retriever Mix"},
                    {"breed": "Chesapeake Bay Retriever"},
                    {"breed": "Newfoundland"},
                ]},
                {"$and": [
                    {"age_upon_outcome_in_weeks": {"$gt": 26.0}},
                    {"age_upon_outcome_in_weeks": {"$lt": 156.0}},
                ]}
            ]
        }
    # Mountain Dog filter
    elif filter_type == "Mountain":
        query = {
            "animal_type": "Dog",
            "sex_upon_outcome": "Intact Male",
            "$and": [
                {"$or": [
                    {"breed": "German Shepherd"},
                    {"breed": "Alaskan Malamute"},
                    {"breed": "Old English Sheepdog"},
                    {"breed": "Siberian Husky"},
                    {"breed": "Rottweiler"},
                ]},
                {"$and": [
                    {"age_upon_outcome_in_weeks": {"$gt": 26.0}},
                    {"age_upon_outcome_in_weeks": {"$lt": 156.0}},
                ]}
            ]
        }
    # Disaster Dog filter
    elif filter_type == "Disaster":
        query = {
            "animal_type": "Dog",
            "sex_upon_outcome": "Intact Male",
            "$and": [
                {"$or": [
                    {"breed": "Doberman Pinscher"},
                    {"breed": "German Shepherd"},
                    {"breed": "Golden Retriever"},
                    {"breed": "Bloodhound"},
                    {"breed": "Rottweiler"},
                ]},
                {"$and": [
                    {"age_upon_outcome_in_weeks": {"$gt": 20.0}},
                    {"age_upon_outcome_in_weeks": {"$lt": 300.0}},
                ]}
                ]
            }
    
    # Pass filtered query to CRUD Module and capture result in dataframe
    df = pd.DataFrame.from_records(db.read(query))
    df.drop(columns=['_id'], inplace=True)
    return df.to_dict('records')

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    # Handle no view data selection
    if viewData is None or len(viewData) == 0:
        return [html.Div("No data available to display on graph.")]
    
    # Update dataframe based on viewData
    df = pd.DataFrame.from_dict(viewData)
    # Return updated dataframe as a pie chart
    return [
        dcc.Graph(figure=px.pie(df, names='breed', title='Preferred Animals'))
    ]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    
    # handle no columns selected
    if selected_columns is None:
        return []
    
    # Return updated style based on column selected
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    # verify viewData is not empty
    if viewData is None or len(viewData) == 0:
        return [
            html.Div("No data available to display on the map.")
        ]
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Verify index is not None or out of range
    if index is None or len(index) == 0 or index[0] >= len(dff):
        return [
            html.Div("No row selected or selected row is out of range.")
        ]
    
    # Because we only allow single row selection, the list can 
    # be converted to a row index here
    row = index[0]
    

    # Austin TX is at [30.75,-97.48]
    # Return updated map
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
               center=[30.75,-97.48], zoom=10, children=[
                   dl.TileLayer(id="base-layer-id"),
                   # Marker with tool tip and popup
                   # Column 13 and 14 define the grid-coordinates for 
                   # the map
                   # Column 4 defines the breed for the animal
                   # Column 9 defines the name of the animal
                   dl.Marker(position=[dff.iloc[row,13], dff.iloc[row,14]],
                             children=[
                                 dl.Tooltip(dff.iloc[row,4]),
                                 dl.Popup([
                                     html.H1("Animal Name"),
                                     html.P(dff.iloc[row,9])
                                 ])
                             ])
               ])
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:28261/
